## Description :

✔️ The idea is to create a Graph with a supervision LLM, that will orchestre the Outher Workers(Agents).

✔️ The Cool here is we will bind This LLM with Agents Not Tools 🤩

## Setup :

curl -fsSL https://ollama.com/install.sh | sh

ollama pull llama3-groq-tool-use:8b

ollama pull llama3.1:8b-instruct-fp16

ollama pull mistral-nemo:12b-instruct-2407-fp16       

ollama pull codegeex4:latest  `Not working with Function calling , but is good at Code related tasks`

ollama pull nomic-embed-text  `For Embedding`



\n

## Build LLMs and Tools :

### Tools :

In [ ]:
from langchain_community.retrievers import TavilySearchAPIRetriever
import os

os.environ["TAVILY_API_KEY"] = "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"
Tavily = TavilySearchAPIRetriever(k=5,include_images = True,include_raw_content = True,search_depth = 'advanced')
#print(Tavily.invoke("What is Langraph ?"))

In [ ]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

Python_Interp = PythonREPL()

### LLMs :

#### Coder

In [ ]:
# Coder LLM:

#from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI


class Coder_Output(BaseModel):
    Code: str = Field(...,description="This is an optimized ,accurate  and Correct PYthon Code")


# LLM with Structered Output
llm = ChatOpenAI(api_key="ollama",model="llama3.1:8b-instruct-fp16",base_url="http://localhost:11434/v1")
structured_llm_coder = llm.with_structured_output(Coder_Output)

# Prompt
system = """You are an expert at generating python Code to response a User Question.\n
Your Code Should be Complete with all used libraries.\n
Your Code should be correct and Optimized as possible.\n
Give just the Code Don't add any additional details or Explainations."""

coder_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "The User question here :\n {question}"),
    ]
)
  
Coder = coder_prompt | structured_llm_coder

- Test

In [ ]:
Question = "Code To desplay a matplotlib graph of cosin math function ?"
input = {"question":Question}
print(Coder.invoke(input).Code)

#### Researcher :

In [ ]:
# Researcher LLM:
from langchain_openai import ChatOpenAI


# LLM with Structered Output
llm = ChatOpenAI(api_key="ollama",model="llama3.1:8b-instruct-fp16",base_url="http://localhost:11434/v1")

# Prompt
system = """You are an expert at using web Search content to respond a User question.\n
Your Response Should be optimazed concise and Clear.\n
Respond just to the User Question Don't add any details or Explanaition, keep the reponse concise."""

Resercher_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "The User question here :\n {question}\n\n Web Search content here : \n {WebResults}"),
    ]
)
  
Resercher = Resercher_prompt | llm

- Test:

In [ ]:
Question = "Remplissage Barrage Alwahda aujourd'hui ? "
Results = Tavily.invoke(Question)
Context = "\n\n-----\n\n".join(result.page_content for result in Results)
input = {"question":Question,"WebResults":Context}
print(Context)
print("\n\n-----------------------------------------------\n\n")
print(Resercher.invoke(input).content)

#### Superviser

In [ ]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI


class Superviser_Output(BaseModel):
    next: Literal["Coder", "Researcher", "FINISH"] = Field(..., description="This is the name of the next worker")


# LLM with Structured Output
llm = ChatOpenAI(api_key="ollama", model="mistral-nemo:12b-instruct-2407-fp16", base_url="http://localhost:11434/v1")
structured_llm_Superviser = llm.with_structured_output(Superviser_Output)

# Prompt
system = """You are a supervisor tasked with managing a conversation between the
    following workers:\n\n 
        Researcher : A worker that Search in the net.
        Coder      : A worker the generate code.
        FINISH     : key word To finish the execution.
        \n\n
    Given a user request respond with the Name of worker to act next. \n
    Each worker will perform a task and respond with his results. \n
    Your Response should be just the Name of the worker from here [Researcher,Coder,FINISH] No additional details or Explanations.\n
    When finished, respond with FINISH."""


Superviser_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
)
options = ["Researcher", "Coder","FINISH"]
Superviser = Superviser_prompt | structured_llm_Superviser


- Test

## Initialize The Graph:

In [ ]:
import operator
from typing import Sequence, TypedDict,Annotated
from langgraph.graph import END, StateGraph, START
from langchain_core.messages import BaseMessage, HumanMessage,ToolMessage,AIMessage

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next:str


graph_builder = StateGraph(State)


## Nodes Functions :

In [ ]:
from Colors import BLUE,RED,YELLOW,GREEN,RESET

### Tools :

### LLMs:

In [ ]:
def Resercher_function(state:State):
    print("State messages :",state["messages"])
    Question = state["messages"][0]
    Results = Tavily.invoke(Question)
    Context = "\n\n-----\n\n".join(result.page_content for result in Results)
    input = {"question":Question,"WebResults":Context}
    print(f"💡{GREEN}Resercher is Thinking !{RESET}")
    report = Resercher.invoke(input)
    return {"messages":report}

In [ ]:
def Coder_function(state:State):
    print("State messages :",state["messages"])
    Question = state["messages"][0]
    input = {"question":Question}
    print(f"💡{GREEN}Coder is Thinking !{RESET}")
    code = Coder.invoke(input)
    return {"messages":report}

In [ ]:
Num_Trials=10

In [ ]:
def Superviser_function(state:State):
    print("State messages :",state["messages"])
    conversation = state["messages"]
    input = {"messages":conversation,"options":options}
    print(f"💡{GREEN}Superviser is Thinking !{RESET}")
    for i in range(Num_Trials):
        print(f"💡{GREEN}Superviser {i} is Thinking !{RESET}")
        response = Superviser.invoke(input)
        if response:
            return {"messages":response,"next":response.next}

In [ ]:
def Choose(state:State):
    return state["next"]  

## Design the Graph:

### Build :

In [ ]:
graph_builder.add_node("Researcher",Resercher_function)
graph_builder.add_node("Coder",Coder_function)
graph_builder.add_node("Superviser",Superviser_function)

In [ ]:
graph_builder.add_edge(START,"Superviser")
graph_builder.add_edge("Coder","Superviser")
graph_builder.add_edge("Researcher","Superviser")
graph_builder.add_conditional_edges("Superviser",Choose,{"Coder":"Coder", "Researcher":"Researcher", "FINISH":"__end__"})

In [ ]:
graph = graph_builder.compile()

### Visualize :

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph(xray= True).draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

## Interact with The Graph :

In [ ]:
from langchain_core.messages import AIMessage

My_Quetion = "What is a node in Langraph?"

for event in graph.stream({"messages": [("user", My_Quetion)]}):
    for value in event.values():
        print(value)